In [1]:
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import time
import random
import pandas as pd
import re
from PIL import Image 
import os 
import PIL 
import glob

Bureau d'étude pour une fonction plus condensée

In [6]:
#cette phrase est écrite jeudi 10 à 10h du matin
def scrap_pages():
    #Start time
    start_time = datetime.now()
    datetime_1 = start_time.strftime("%Y-%m-%d_%Hh%M")

    #site to scrap
    source = 'as24'

    #log update
    log_import = pd.read_csv('log.csv')
    log_new = pd.DataFrame({'source' : [source],
                            'step' : ['scrap pages'],
                            'status' : ['started'],
                            'time' : [datetime.now()],
                            'details' : [""]})
    log = log_import.append(log_new , ignore_index=True)
    log.to_csv('log.csv', index=False)

    #fonction récurrente : extraction du nombre
    def extract_int(sample):
        list = re.findall(r'\d+', sample)
        while True:
            try:
                res = int("".join(map(str, list)))
                break
            except ValueError:
                res = np.nan
                break
        return res

    #identifier le nombre de résultats et de pages d'une url listing d'annonces
    def url_nb_results(url):
        resp_single = requests.get(url)
        soup_ = BeautifulSoup(resp_single.content, 'html.parser')
        nb_offres = soup_.find('div', class_='cl-filters-summary__results-label sc-font-l sc-hidden-m sc-hidden-s sc-grid-row').find('span').text
        return extract_int(nb_offres)
    
    def url_nb_pages(url):
        nb_offres = url_nb_results(url)
        if nb_offres%20>0: reste = 1
        else: reste = 0
        nb_pages = (nb_offres//20) + reste
        return nb_pages

    #Création du DataFrame avec tous les url de listing
    #============================================================================
    #=== pattern pour trouver l'erreur : à supprimer ===
    print('pattern 1 : avant la boucle')
    #=== pattern pour trouver l'erreur : à supprimer ===
    url = 'https://www.autoscout24.fr/lst-moto/?sort=price&desc=1&ustate=N%2CU&size=20&page=1&cy=F&priceto=99000&atype=B&fc=1&qry=&recommended_sorting_based_id=16f9d2b5-588d-9c51-e053-0100007f13ed&'
    df_listing = pd.DataFrame({"url": [np.nan]})
    url_row = {"url": ""}
    nb_results = url_nb_results(url)
    print(f'Il y a en tout {nb_results} annonces')
    boucle = 0
    try:
        while nb_results>400:
            boucle +=1
            print('boucle numéro : ', boucle, '(de 20 url de 20 annonces)')
            time.sleep(2)

            #création du df url de listing de 20 pages : df_listing
            for page in range(1, 21):
                url_row['url'] = url.split('page=')[0] + 'page=' + str(page) + '&cy=' + url.split('&cy=')[1]
                df_listing = df_listing.append(url_row, ignore_index=True)

            #extraire le dernier prix du listing
            time.sleep(1)
            url_listing_last_price = df_listing.iloc[len(df_listing)-1,0]

            #catch de la liste d'annonces light de la dernière url de la boucle en cours pour extraction dernier prix : 
            resp_single = requests.get(url_listing_last_price)
            soup_ = BeautifulSoup(resp_single.content, 'html.parser')
            announces_list = soup_.find_all("div", class_="cl-list-element cl-list-element-gap")
            #=== print indicatif ===
            print('la dernière des 20 pages contient ', len(announces_list), ' annonces')
            #=== print indicatif ===
            time.sleep(2)
            price = extract_int(announces_list[19].find("div", class_="cldt-summary-payment").find("span", class_="cldt-price sc-font-xl sc-font-bold").text)

            #création de la nouvelle url
            url = url.split('priceto=')[0] + 'priceto=' + str(price) + '&atype=' + url.split('&atype=')[1]
            url = url.split('&fc=')[0] + '&fc=' + str(boucle) + '&qry=&' + url.split('&qry=&')[1]

            #=== ajout du corps de la fonction pour voir le pb : supprimer pour ne laisser que la fonction ===
            nb_results = url_nb_results(url)
            print('pour la prochaine boucle il reste : ', nb_results, 'résultats')
            #=== ajout du corps de la fonction pour voir le pb : supprimer pour ne laisser que la fonction ===


        #si on arrive à la dernière page dans laquelle il y a moins de 20 résultats :
        else:
            print('dernière boucle')
            for page in range(1, url_nb_pages(url)):
                url_row['url'] = url.split('page=')[0] + 'page=' + str(page) + '&cy=' + url.split('&cy=')[1]
                df_listing = df_listing.append(url_row, ignore_index=True)

        df_listing = df_listing.drop(0)
        #============================================================================
        
        print('df_listing terminé : création des fichiers')
        #url to scrap : stockage html de chaque listing d'annonces light dans des fichiers
        for i in range(len(df_listing)-1):
            url = df_listing.iloc[i,0]
            response = requests.get(url)
            file_name = source + "-" + str(i) + "-" + datetime_1

            with open(f"pages/{file_name}.html", "w")  as file:
                file.write(response.text)
                file.close()

        #End time
        end_time = datetime.now()
        td = end_time - start_time

        #log update
        log_import = pd.read_csv('log.csv')
        log_new = pd.DataFrame({'source' : [source],
                                'step' : ['scrap pages'],
                                'status' : ['completed'],
                                'time' : [datetime.now()],
                                'details' : [f"{td.seconds/60} minutes elapsed, {len(df_listing)} pages scrapped"]})
        log = log_import.append(log_new , ignore_index=True)
        log.to_csv('log.csv', index=False)
        print('over')
    
    except (ValueError, TypeError, NameError, KeyError, RuntimeWarning, IndexError) as err:
    #log update
      
        log_import = pd.read_csv('log.csv')
        log_new = pd.DataFrame({'source' : [source],
                                'step' : ['scrap pages'],
                                'status' : ['error'],
                                'time' : [datetime.now()],
                                'details' : [err]})
        log = log_import.append(log_new , ignore_index=True)
        log.to_csv('log.csv', index=False)
    
#     return df_listing

In [7]:
scrap_pages()

pattern 1 : avant la boucle
Il y a en tout 3803 annonces
boucle numéro :  1 (de 20 url de 20 annonces)
la dernière des 20 pages contient  20  annonces
pour la prochaine boucle il reste :  3415 résultats
boucle numéro :  2 (de 20 url de 20 annonces)
la dernière des 20 pages contient  20  annonces
pour la prochaine boucle il reste :  3020 résultats
boucle numéro :  3 (de 20 url de 20 annonces)
la dernière des 20 pages contient  20  annonces
pour la prochaine boucle il reste :  2645 résultats
boucle numéro :  4 (de 20 url de 20 annonces)
la dernière des 20 pages contient  20  annonces
pour la prochaine boucle il reste :  2246 résultats
boucle numéro :  5 (de 20 url de 20 annonces)
la dernière des 20 pages contient  20  annonces
pour la prochaine boucle il reste :  1873 résultats
boucle numéro :  6 (de 20 url de 20 annonces)
la dernière des 20 pages contient  20  annonces
pour la prochaine boucle il reste :  1474 résultats
boucle numéro :  7 (de 20 url de 20 annonces)
la dernière des 20 pa

Essai avec la dernière sauvegarde sur Notion : jeudi matin (reprise de la dernière version de mardi soir)

In [27]:
#le problème est au niveau de la request qui ne marche pas pour autoscout donc peut que ca soit bloqué par le site lui meme
# mais ca ne marche pas pour les autres sites non plus